In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.experimental.numpy as tnp
import glob

print("Using TensorFlow version %s" % tf.__version__)

Using TensorFlow version 2.7.0


In [2]:
tnp.experimental_enable_numpy_behavior()

In [3]:
tf.test.is_built_with_cuda()

True

In [4]:
tf.config.list_physical_devices('GPU')

2021-12-05 11:15:09.631523: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-05 11:15:09.635655: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-05 11:15:09.635840: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
from tensorflow.keras.layers import LSTM,Concatenate, Dense, Flatten ,Activation ,Input , BatchNormalization,Dropout , Bidirectional
from tensorflow.keras.models import Sequential, Model

In [6]:
print("Eager execution: {}".format(tf.executing_eagerly()))

Eager execution: True


In [7]:
building1_filenames = set(glob.glob("../datasets/building1/*/*.feather"))
building2_filenames = set(glob.glob("../datasets/building2/*/*.feather"))
building3_filenames = set(glob.glob("../datasets/building3/*/*.feather"))
training_filenames = set(glob.glob("../datasets/building1/known/7*.feather"))
#test1_filenames = building1_filenames - training_filenames

In [8]:
import random
seed=100
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

In [9]:
# generate training samples

dfs = []
for file in training_filenames:
    df = pd.read_feather(file)
    dfs.append(df)


training_data = pd.concat(dfs, ignore_index=True)
training_data["iphoneAccX"] = -1*training_data["iphoneAccX"]
training_data["iphoneAccY"] = -1*training_data["iphoneAccY"]
training_data["iphoneGyroX"] = -1*training_data["iphoneGyroX"]
training_data["iphoneGyroY"] = -1*training_data["iphoneGyroY"]
training_data["iphoneMagX"] = -1*training_data["iphoneMagX"]
training_data["iphoneMagY"] = -1*training_data["iphoneMagY"]
#training_data.head()

orientation_acc = training_data[["iphoneAccX", "iphoneAccY", "iphoneAccZ"]]
orientation_gyro = training_data[["iphoneGyroX", "iphoneGyroY", "iphoneGyroZ"]]
orientation_mag = training_data[["iphoneMagX", "iphoneMagY", "iphoneMagZ"]] 

#orientation_data.head()

In [10]:
orientation_target = training_data[["orientX", "orientY", "orientZ", "orientW"]] 
orientation_target.head()

,orientX,orientY,orientZ,orientW
0,0.025497,-0.010281,0.000252,0.999622
1,0.025480,-0.010309,0.000230,0.999622
2,0.025527,-0.010327,0.000246,0.999621
3,0.025486,-0.010316,0.000241,0.999622
4,0.025504,-0.010333,0.000285,0.999621


In [11]:
acc_np = orientation_acc.to_numpy()
gyro_np = orientation_gyro.to_numpy()
mag_np = orientation_mag.to_numpy()

In [12]:
orient_np = orientation_target.to_numpy()
len(orient_np)

65652

In [13]:
def generate_training_samples(acc_, gyro_, mag_, orient_, batch_size = 64):
    while True:
        xa_batch = np.zeros((batch_size,100,3))
        xg_batch = np.zeros((batch_size,100,3))
        xm_batch = np.zeros((batch_size,100,3))
        y_theta_batch = np.zeros((batch_size,4))
        #y_sigma_batch = np.finfo(np.float32).eps* np.ones((batch_size,6)) ## To remove
        #y_batch = np.concatenate((y_theta_batch, y_sigma_batch), axis=1)
        #print(y_batch.shape)
        
        j = 0
        for i in range(len(orient_)):
            xa_batch[j,:,:] = acc_[i, :]
            xg_batch[j,:,:] = gyro_[i,:]
            xm_batch[j,:,:] = mag_[i,:]
            y_theta_batch[j,:4] = orient_[i,:]
            j += 1
            if j >= batch_size:
                j = 0              
                yield([xa_batch,xg_batch, xm_batch],[y_theta_batch])
            #---------------------------------------------------------------------------------------

In [14]:
checkpoints_path = os.path.join(os.getcwd(),'../datasets/CheckPoints_Model_OrientNet_Building1_File_7')

if not os.path.exists(checkpoints_path):
  os.mkdir(checkpoints_path)
check_point_template_path = os.path.join(checkpoints_path,'ckpt_epoch_{epoch:03d}_loss_{loss:.4f}_.hdf5') # vloss_{val_loss:.4f}
check_point_callback = tf.keras.callbacks.ModelCheckpoint(check_point_template_path)

import re #regular expresion
def get_all_checkpoints(checkpoints_path,checkpoint_main_name = 'ckpt'):
  all_checkpoints = [j for j in os.listdir(checkpoints_path) if j.startswith(checkpoint_main_name)]
  return all_checkpoints

def check_if_available_checkpoints(checkpoints_path,checkpoint_main_name = 'ckpt'):
  all_checkpoints = get_all_checkpoints(checkpoints_path,checkpoint_main_name)
  if(len(all_checkpoints) > 0):#checkpoints avilable
    all_checkpoints.sort(reverse=True)    
    latest_check_point = all_checkpoints[0]
    initial_epoch = int(re.search('epoch_(.*?)_', latest_check_point).group(1))    
  else:
    latest_check_point = None
    initial_epoch = 0
    
  return initial_epoch , latest_check_point


# Check if there are any check points initially
initial_epoch , latest_check_point = check_if_available_checkpoints(checkpoints_path)

In [15]:
orient_learning_rate = 0.0005 # convergence  within  20  epochs \ref{IDOL}

In [16]:
import tensorflow_graphics.geometry.transformation as tfg
#def quaterinion_inv(q):
#    return (1/(tnp.sum(q**2))) * tnp.array([-1*q[0], -1*q[1], -1*q[2], q[3]])
#
#def log_bar(q, batch_size = 64):
#    #print(tf.shape(q[3]))
#    log_batch = tnp.zeros((batch_size,3))
#    log_list = tf.unstack(log_batch)
#
#    for i in range(batch_size):
#        if q[3][i] == 0:
#            log_list[i] =tnp.array([0.0, 0.0, 0.0], dtype=tnp.float32)
#            #print(tf.shape(log_batch[i]))
#        else:
#            v = tf.slice(q,
#                   begin=[i,0],
#                   size=[3,1])
#            log_list[i] = tnp.arctan2(tnp.sum(tnp.square(v)), q[3][i])/tnp.sum(tnp.square(v)) * v
#    return tf.stack(log_list)
#
#
#def quaternion_multiply(quaternion1, quaternion0):
#    #w0, x0, y0, z0 = quaternion0 # ERROR with TF
#    #w1, x1, y1, z1 = quaternion1
#    w0 = quaternion0[3]
#    x0 = quaternion0[0]
#    y0 = quaternion0[1]
#    z0 = quaternion0[2]
#    w1 = quaternion1[3]
#    x1 = quaternion1[0]
#    y1 = quaternion1[1]
#    z1 = quaternion1[2]
#    return tnp.array([-x1 * x0 - y1 * y0 - z1 * z0 + w1 * w0,
#                     x1 * w0 + y1 * z0 - z1 * y0 + w1 * x0,
#                     -x1 * z0 + y1 * w0 + z1 * x0 + w1 * y0,
#                     x1 * y0 - y1 * x0 + z1 * w0 + w1 * z0])
#
#def boximinus(q1, q2):
#    return 2 * log_bar(quaternion_multiply(quaterinion_inv(q2), q1))

In [17]:
# Loss Function
class OreintLoss(tf.keras.losses.Loss):
  def call(self, y_true, y_pred):
    # custom loss L_orient = 0.5*(q [-] \hat{q})^T *Sig^{-1}* 
    y_true = tf.cast(y_true, y_pred.dtype)
    
    #q = y_true[:, 0:4]                   ##y_true[0:4][:]
    q = tf.slice(y_true,
                    begin=[0,0],
                    size=[-1,4])  


    #q_est = y_pred[:, 0:4]   #########y_pred[0:4][:]
    q_est = tf.slice(y_pred,
                    begin=[0,0],
                    size=[-1,4])  


    s_00 = tf.math.exp(tf.reshape(tf.slice(y_pred,  #
           begin=[0,4],
           size=[-1,1]), [-1]))
    s_11 = tf.math.exp(tf.reshape(tf.slice(y_pred,  #
           begin=[0,5],  
           size=[-1,1]), [-1]))
    s_22 = tf.math.exp(tf.reshape(tf.slice(y_pred, #
           begin=[0,6],
           size=[-1,1]), [-1]))
    s_01 = tf.math.abs(tf.math.multiply(tf.reshape(tf.slice(y_pred,
           begin=[0,7],  
           size=[-1,1]), [-1]), tf.math.sqrt(tf.math.multiply(s_00, s_11))))
    s_02 = tf.math.abs(tf.math.multiply(tf.reshape(tf.slice(y_pred, 
           begin=[0,8],    
           size=[-1,1]), [-1]), tf.math.sqrt(tf.math.multiply(s_00, s_22))))
    s_12 = tf.math.abs(tf.math.multiply(tf.reshape(tf.slice(y_pred,
           begin=[0,9],  
           size=[-1,1]), [-1]), tf.math.sqrt(tf.math.multiply(s_11, s_22))))

           
    sig0 = tf.stack([s_00, s_01, s_02], axis=1)
    sig1 = tf.stack([s_01, s_11, s_12], axis=1)
    sig2 = tf.stack([s_02, s_12, s_22], axis=1)
    sig = tf.stack([sig0, sig1, sig2], axis=2)

    i = tfg.quaternion.inverse(q_est)

    mult_ = tfg.quaternion.multiply(i, q)
    log_ = tf.slice(mult_,
                    begin=[0,0],
                    size=[-1,3])
    omega=tf.slice(mult_,
              begin=[0,3],
              size=[-1,1])

 
    log_ = tf.linalg.normalize(log_, ord='euclidean', axis=None, name=None)
    atan_ = tf.atan(tf.divide(log_[1], omega))
    v = log_[0]
    z = tf.zeros_like(v)
    v = tf.where(tf.equal(v, z), tf.zeros_like(v), v)


    
    delta = tf.math.scalar_mul(2.0, tf.multiply(v, atan_))                  #boximinus(q, q_est)
    delta = tf.expand_dims(delta, -1)


    #sig = tf.eye(3, batch_shape=[64])#tnp.ones((64,3,3), dtype=tnp.float32)
    #sig[:,0, 0] = y_pred[:,4]
    #sig[:,1, 1] = y_pred[:,5]
    #sig[:,2, 2] = y_pred[:,6]
    #sig[:,0, 1] = y_pred[:,7]
    #sig[:,1, 0] = y_pred[:,7]
    #sig[:,0, 2] = y_pred[:,8]
    #sig[:,2, 0] = y_pred[:,8]
    #sig[:,1, 2] = y_pred[:,9]
    #sig[:,2, 1] = y_pred[:,9]
    

    
    e = tf.math.scalar_mul(tf.keras.backend.epsilon() ,tf.eye(3, batch_shape=[64]))

    sig_inv = tf.linalg.inv(tf.add(sig,e)) #  tf.add(sig,e)
    m_ = tf.matmul(sig_inv, delta)
    m = tf.matmul(delta, m_, transpose_a=True)


    l_s = tf.linalg.logdet(sig)

    l = tf.math.scalar_mul(0.5, m)  + tf.math.scalar_mul(0.00000000005, l_s) # 0.00000000005 weird factor that worked


    return tf.reduce_mean(l)


class MyLossP(tf.keras.losses.Loss):
  def call(self, y_true, y_pred):
    # y_pred = tf.convert_to_tensor_v2(y_pred)
    # y_true = tf.cast(y_true, y_pred.dtype)
    return tf.sqrt(tf.reduce_mean((y_pred - y_true)**2, axis=-1))





In [18]:
class OreintNet():
    def build_model(self):
        # Building Network
        initializer = tf.keras.initializers.GlorotUniform()
        #1. Define inputs
        Acc_input = Input(shape=(100,3), batch_size=64 , name = 'Acc_input')    
        Gyro_input = Input(shape=(100,3),batch_size=64   , name = 'Gyro_input')
        Mag_input = Input(shape=(100,3), batch_size=64   , name = 'Mag_input')
        
        MergedLayer = Concatenate()([Acc_input , Gyro_input, Mag_input])
        #
        ##MergedLayer = Input(shape=(9) , name = 'imu_input')
        #2. LSTM
        LSTM1 = (LSTM(100, kernel_initializer = 'glorot_uniform', recurrent_initializer = 'orthogonal', return_sequences=True))(MergedLayer) # , return_sequences=True
        LSTM2 = (LSTM(100, kernel_initializer = 'glorot_uniform', recurrent_initializer = 'orthogonal', return_sequences=False))(LSTM1)
        #print("Output shape of the LSTM layer's output : ", LSTM2.shape)
        
        #3. Fully-Connected (Theta)
        Dense1 = Dense(units=100, input_shape=(64,100), activation='tanh')(LSTM2)
        Dense2 = Dense(units=32, activation='tanh')(Dense1)
        theta_output = Dense(units=4, name='theta_out')(Dense2) # Sigma Outputs
        
        #4. Fully-Connected (Sigma)
        Dense3 = Dense(units=100, input_shape=(64,100), activation='tanh')(LSTM2) # input_shape=(
        Dense4 = Dense(units=32, activation='tanh')(Dense3)
        sigma_output = Dense(units=6, activation='sigmoid', name='sig_out')(Dense4) # Theta Outputs  activation='linear'


        output = Concatenate()([theta_output, sigma_output])
        #5. Define and compile The model
        Network = Model([Acc_input,Gyro_input, Mag_input],  [output])  #  [Acc_input,Gyro_input, Mag_input]
        Network.compile(loss=OreintLoss(), optimizer=tf.keras.optimizers.Adam(learning_rate=orient_learning_rate)) #loss=OreintLoss()
        return Network
        

In [19]:
#class OreintNet(Model):
    

In [20]:
network_builder = OreintNet()
Nepochs = 20
if initial_epoch > 0:
  print('continue after epoch' , initial_epoch , ' - checkpoint: ',latest_check_point,' epochs = ',Nepochs)
  orientation_network = tf.keras.models.load_model(os.path.join(checkpoints_path,latest_check_point),compile=False)
  orientation_network.compile(loss=OreintLoss(), optimizer=tf.keras.optimizers.Adam(learning_rate=orient_learning_rate))
else:
  print('start from scratch : epochs = ', Nepochs)
  orientation_network = network_builder.build_model()

start from scratch : epochs =  20


2021-12-05 11:15:11.212534: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-05 11:15:11.213297: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-05 11:15:11.213734: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-05 11:15:11.213970: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [21]:
orientation_network.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Acc_input (InputLayer)         [(64, 100, 3)]       0           []                               
                                                                                                  
 Gyro_input (InputLayer)        [(64, 100, 3)]       0           []                               
                                                                                                  
 Mag_input (InputLayer)         [(64, 100, 3)]       0           []                               
                                                                                                  
 concatenate (Concatenate)      (64, 100, 9)         0           ['Acc_input[0][0]',              
                                                                  'Gyro_input[0][0]',         

In [22]:

history = orientation_network.fit(
    generate_training_samples(acc_np, gyro_np, mag_np, orient_np),
    epochs=Nepochs,
    initial_epoch=initial_epoch,
    steps_per_epoch=len(orient_np),
    batch_size=64,
    callbacks = [check_point_callback]
)

Epoch 1/20


2021-12-05 11:15:15.531181: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8201
2021-12-05 11:15:16.002293: I tensorflow/core/util/cuda_solvers.cc:179] Creating GpuSolver handles for stream 0x7d0b940


    8/65652 [..............................] - ETA: 18:22 - loss: nan   

2021-12-05 11:15:16.505834: W tensorflow/core/kernels/linalg/cholesky_op_gpu.cu.cc:202] Cholesky decomposition was not successful for batch 0. The input might not be valid. Filling lower-triangular output with NaNs.
2021-12-05 11:15:16.505872: W tensorflow/core/kernels/linalg/cholesky_op_gpu.cu.cc:202] Cholesky decomposition was not successful for batch 1. The input might not be valid. Filling lower-triangular output with NaNs.
2021-12-05 11:15:16.505881: W tensorflow/core/kernels/linalg/cholesky_op_gpu.cu.cc:202] Cholesky decomposition was not successful for batch 2. The input might not be valid. Filling lower-triangular output with NaNs.
2021-12-05 11:15:16.505887: W tensorflow/core/kernels/linalg/cholesky_op_gpu.cu.cc:202] Cholesky decomposition was not successful for batch 3. The input might not be valid. Filling lower-triangular output with NaNs.
2021-12-05 11:15:16.505892: W tensorflow/core/kernels/linalg/cholesky_op_gpu.cu.cc:202] Cholesky decomposition was not successful for ba

InvalidArgumentError:  Input is not invertible.
	 [[node OreintLoss/MatrixInverse
 (defined at /tmp/ipykernel_27865/1661278321.py:82)
]] [Op:__inference_train_function_6355]

Errors may have originated from an input operation.
Input Source operations connected to node OreintLoss/MatrixInverse:
In[0] OreintLoss/Add:

Operation defined at: (most recent call last)
>>>   File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
>>>     return _run_code(code, main_globals, None,
>>> 
>>>   File "/usr/lib/python3.8/runpy.py", line 87, in _run_code
>>>     exec(code, run_globals)
>>> 
>>>   File "/home/nav-shaza/Desktop/Re-IDOL/.venv/lib/python3.8/site-packages/ipykernel_launcher.py", line 16, in <module>
>>>     app.launch_new_instance()
>>> 
>>>   File "/home/nav-shaza/Desktop/Re-IDOL/.venv/lib/python3.8/site-packages/traitlets/config/application.py", line 846, in launch_instance
>>>     app.start()
>>> 
>>>   File "/home/nav-shaza/Desktop/Re-IDOL/.venv/lib/python3.8/site-package

In [ ]:
pos_learning_rate = 0.001